In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import stats

from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

%precision 3

%matplotlib inline

# 2집단 데이터에 대한 t검정

## 대응표본 t검정 ( Paired t-test )




* 비교하려는 두 집단이 서로 독립적이지 않을 때, 한 집단이 두번 테스트 되었을 때 사용 (dependent sample)


* 책의 예시에서는 동일한 집단이 약을 먹기전과 먹은 후 체온을 비교해서 약이 효과가 있는지 검정하는 것이므로 paired t-test 사용


* 귀무가설 : 차잇값 = 0  vs 대립가설: 차잇값 != 0 (양측검정)








In [4]:
paired_test_data=pd.read_csv('3-9-1-paired-t-test.csv')
print(paired_test_data)

  person medicine  body_temperature
0      A   before              36.2
1      B   before              36.2
2      C   before              35.3
3      D   before              36.1
4      E   before              36.1
5      A    after              36.8
6      B    after              36.1
7      C    after              36.8
8      D    after              37.1
9      E    after              36.9


In [7]:
before = paired_test_data.query('medicine =="before"')["body_temperature"]
after = paired_test_data.query('medicine == "after"')["body_temperature"]

before = np.array(before)
after = np.array(after)

diff = after - before
diff

array([ 0.6, -0.1,  1.5,  1. ,  0.8])

In [8]:
stats.ttest_1samp(diff,0)  # P-value 가 0.044 < 0.05 이므로 귀무가설 기각

Ttest_1sampResult(statistic=2.901693483620596, pvalue=0.044043109730074276)

In [9]:
stats.ttest_rel(after,before)

Ttest_relResult(statistic=2.901693483620596, pvalue=0.044043109730074276)

* stats.ttest_rel(a,b) : Related된 두 샘플을 Two Side t검정할 때 사용, a와 b는 array-like

![t값](https://cdn.smassets.net/assets/cms/cc/uploads/t-test-formulas-t-tests-explained.png)


* Paired T-test는 두 샘플의 차이값을 구한 뒤에 One Sample T-test를 하는 것과 동일하다.

## 독립표본 t검정

* 독립표본 t검정은 서로 독립된 두 샘플 간 차이를 검정하기 위한 t검정이다

* 독립표본 t검정의 t값은 다음과 같다

![t값](http://www.stat.yale.edu/Courses/1997-98/101/tstat2.gif)

In [11]:
mean_bef = sp.mean(before)
mean_aft = sp.mean(after)

sigma_bef = sp.var(before , ddof = 1)
sigma_aft = sp.var(after, ddof = 1)

m = len(before)
n = len(after)

t_value = (mean_aft-mean_bef)/sp.sqrt((sigma_bef/m+sigma_aft/n))

t_value
                                

3.1557282344421034

In [12]:
stats.ttest_ind(after, before, equal_var = False)

# P-value = 0.013 < 0.05 이므로 귀무가설 기각

Ttest_indResult(statistic=3.1557282344421034, pvalue=0.013484775682079892)

* stats.ttest_ind(a,b) : 두 independent한 샘플의 Two side t검정을 할 때 사용, a와 b는 array-like 

* equal_var = True 이면 등분산 가정

* 등분산 가정일 경우 t값

![t값](https://www.boost.org/doc/libs/1_46_0/libs/math/doc/sf_and_dist/equations/dist_tutorial1.png)

## 분할표 검정 (contingency table)

* 카이제곱 검정(chi-squared test) : 관찰된 빈도가 기대되는 빈도와 의미있게 다른지에 여부를 검증하기 위해 사용되는 검증방법


* 귀무가설: 기대도수와 관측도수가 일치한다 vs 대립가설 : 기대도수와 관측도수가 일치하지 않는다

In [13]:
1-sp.stats.chi.cdf(x=6.667, df=1)

2.6108559758597494e-11

* 카이제곱분포는 좌우대칭이 아니므로 (1-alpha)*2를 이용하지 않음 , t검정(200p)와 다른점


* 카이제곱 검정은 단측검정만 존재

![카이제곱분포](http://www.ktword.co.kr/img_data/2365_4.JPG)

In [14]:
click_data = pd.read_csv('3-10-1-click_data.csv')

print(click_data)

  color  click  freq
0  blue  click    20
1  blue    not   230
2   red  click    10
3   red    not    40


In [17]:
cross = pd.pivot_table(
    data = click_data,
    values = 'freq',
    aggfunc = 'sum',
    index = 'color',
    columns = 'click'
)

print(cross)

click  click  not
color            
blue      20  230
red       10   40


 * sp.stats.chi2_contingency(observed) :  contingency table로 변수의 독립성을 카이제곱검정

In [18]:
sp.stats.chi2_contingency(cross, correction = False)

# 통계량, P값 , 자유도, 기대도수표 순서

(6.666666666666666, 0.009823274507519247, 1, array([[ 25., 225.],
        [  5.,  45.]]))